In [1]:
import re
import numpy as np
import pandas as pd
from IPython.display import display
from __future__ import print_function, division

### Extracts 2016 & 2017 ownership relationships into single dataframe


In [13]:
def extractOwnership(data):
    dataframes = []

    companies = data[data.id.notnull()]
    row_iterator = companies.iterrows()
    _, last = row_iterator.next()

    for i, row in row_iterator:
        company = last.company.split('*')[0].strip()
        dft = data[_:i]
        dft.columns = ['col0','col1','col2','col3','title','role']
        dft = dft.dropna(axis=0, how='all')
        start = dft[dft.col1=='Owned by:'].index+1
        end = dft[dft.col1=='Has ownership interest in:'].index-1
        if(len(end)>0):
            dfts = dft.loc[start[0]:end[0]]
            dfts_h = dft.loc[(end[0]+2):]
        else:
            dfts = dft.loc[start[0]:]
            
        dfts = dfts.drop(['col0','title'],axis=1)
        
        dfts.col1 = company
        dfts.columns = ['company','percent','owner','role']
        dataframes.append(dfts)
        last = row
        _ = i

    df = pd.concat(dataframes)
    df.owner = df.owner.map(lambda x: str(x).split('*')[0].strip())
    df['year']=year
    return df

dataframes = []
for year in [2016,2017]:
    un = pd.read_csv('raw/trump-oge278e-exhibit-a-'+str(year)+'-raw.csv',header=None)
    un.columns = ['id','company','col2', 'col3', 'title', 'role']
    dataframes.append(extractOwnership(un))
    
entityRelDf = pd.concat(dataframes, ignore_index=True)

### Initial clean up of entity relations

In [14]:
entityRelDf.columns = ['entityA','ownership','entityB','role','year']
entityRelDf.entityA = entityRelDf.entityA.map(lambda x: x.upper())
entityRelDf.entityB = entityRelDf.entityB.map(lambda x: x.upper())

### Combine 2016 & 2017 entities into single dataframe

In [15]:
entity2016 = pd.concat([entityRelDf[entityRelDf.year==2016].entityA,entityRelDf[entityRelDf.year==2016].entityB]).unique()
entity2017 = pd.concat([entityRelDf[entityRelDf.year==2017].entityA,entityRelDf[entityRelDf.year==2017].entityB]).unique()

entity2016 =  pd.DataFrame(entity2016,columns=['entity'])
entity2016['year']=2016

entity2017 =  pd.DataFrame(entity2017,columns=['entity'])
entity2017['year']=2017

entityDf = pd.concat([entity2016,entity2017])

### Add initial org_slug to entities

In [16]:
def slugify(s):
    s = str(s)
    s = re.sub(r'( |,|\.|:|;)', '-', s)
    s = re.sub(r"(#|&|\*|\/|\\|'|’|\(|\)|\?|\!)", '', s)
    s = re.sub(r'-+', '-', s)
    s = re.sub(r'-$', '', s)
    s = s.lower()
    return s

entityRelDf['entityASlug'] = entityRelDf.entityA.map(lambda x: slugify(x))
entityRelDf['entityBSlug'] = entityRelDf.entityB.map(lambda x: slugify(x))

entityDf['slug'] = entityDf.entity.map(slugify)
display(entityDf.sample(5))
print(len(entityDf))

,entity,year,slug
371,TRUMP MARKS PUNTA DEL ESTE LLC,2016,trump-marks-punta-del-este-llc
128,OCEAN DEVELOPMENT MEMBER INC,2017,ocean-development-member-inc
488,DT BALI HOTEL MANAGER MEMBER CORP,2016,dt-bali-hotel-manager-member-corp
318,TRUMP MARKS BATUMI LLC,2016,trump-marks-batumi-llc
316,TRUMP MARKS BAJA CORP,2016,trump-marks-baja-corp


1125


### Standardize slugs & names to match the rest of the database

In [17]:
companyCols = ['org_slug','org_name','other_org_names','source']

companies = []
companies.append(pd.read_csv('data/1_trump-oge278e-part-one-companies.csv', na_values=''))
companies.append(pd.read_csv('data/2_trump-oge278e-part-two-companies.csv', na_values=''))
companies.append(pd.read_csv('data/3_trump-oge278e-exhibit-a-companies.csv', na_values=''))
companies = pd.concat(companies)
companies = companies[companyCols].fillna('')
companies.sample(10)

,org_slug,org_name,other_org_names,source
87,dt-marks-gurgaon-managing-member-corp,DT Marks Gurgaon Managing Member Corp,,2015 Part 1;2016 Part 1;2017 Part 1;2015 Exhib...
5,beach-haven-properties-no-3-inc,Beach Haven Properties No. 3 Inc.,,2015 Exhibit A
98,dt-marks-qatar-llc,DT Marks Qatar LLC,,2015 Part 1;2016 Part 1;2017 Part 1;2015 Exhib...
565,trump-international-development-llc,Trump International Development LLC,,2015 Part 1;2016 Part 1;2017 Part 1;2015 Exhib...
223,thc-qatar-hotel-manager-llc,THC Qatar Hotel Manager LLC,,2015 Part 1;2016 Part 1;2017 Part 1;2015 Exhib...
218,thc-jeddah-hotel-advisor-llc,THC Jeddah Hotel Advisor LLC,,2016 Part 1;2017 Part 1;2016 Exhibit A
361,trump-international-golf-club-lc-trump-interna...,Trump International Golf Club LC (Trump Intern...,Trump International Golf Club LC;Trump Interna...,2015 Part 1;2016 Part 1;2017 Part 1;2015 Part ...
441,trump-marks-philadelphia-llc,Trump Marks Philadelphia LLC,,2015 Part 1;2016 Part 1;2017 Part 1;2015 Exhib...
74,dt-lido-hotel-manager-llc,DT Lido Hotel Manager LLC *,,2016 Part 1;2017 Part 1;2016 Exhibit A
151,jupiter-golf-club-managing-member-corp,Jupiter Golf Club Managing Member Corp,,2015 Part 1;2016 Part 1;2017 Part 1;2015 Part ...


In [18]:
# Santardized the slugs within the Entities dataframe
companies['other_org_names_slugs'] = companies.other_org_names.map(lambda x: map(slugify, '' if pd.isnull(x) else x.split(';')))

match_type = []
std_slugs = []
std_name = []

row_iterator = entityDf.iterrows()
for i, row in row_iterator:
    match = companies[companies.org_slug == row.slug]
    match_other = companies[companies.other_org_names_slugs.map(lambda y:''.join([x for x in y if x == row.slug])) != '']

    # if slugs match org_slug from the companies table
    if(len(match)>0):
        std_name.append(match.org_name.iloc[0])
        std_slugs.append(match.org_slug.iloc[0])
        match_type.append('match')
        
    # if is in other_org_names list from the companies table    
    elif(len(match_other)==1):
        std_name.append(match_other.org_name.iloc[0])
        std_slugs.append(match_other.other_org_names_slugs.map(lambda y:''.join([x for x in y if x == row.slug])).iloc[0])
        match_type.append('other')

    # if not keep slug as org_slug    
    else:       
        std_name.append(row.entity)
        std_slugs.append(row.slug)        
        match_type.append('new')


entityDf['org_name'] = std_name
entityDf['org_slug'] = std_slugs
entityDf['match_type'] = match_type

display(entityDf.sample(10))

match = entityDf[entityDf.match_type == 'match']
other = entityDf[entityDf.match_type == 'other']
new = entityDf[entityDf.match_type == 'new']


,entity,year,slug,org_name,org_slug,match_type
3,40 WALL STREET COMMERCIAL LLC,2017,40-wall-street-commercial-llc,40 Wall Street Commercial LLC,40-wall-street-commercial-llc,match
38,DJT AEROSPACE LLC,2016,djt-aerospace-llc,DJT Aerospace LLC,djt-aerospace-llc,match
293,TRUMP INTERNATIONAL DEVELOPMENT MEMBER CORP,2016,trump-international-development-member-corp,Trump International Development Member Corp,trump-international-development-member-corp,match
274,TRUMP DRINKS ISRAEL MEMBER CORP,2017,trump-drinks-israel-member-corp,Trump Drinks Israel Member Corp,trump-drinks-israel-member-corp,match
481,THC CHINA DEVELOPMENT MANAGEMENT CORP,2016,thc-china-development-management-corp,THC China Development Management Corp (fka Tru...,thc-china-development-management-corp,other
324,TRUMP MARKS CANOUAN LLC,2017,trump-marks-canouan-llc,Trump Marks Canouan LLC,trump-marks-canouan-llc,match
83,DT MARKS QATAR MEMBER CORP,2017,dt-marks-qatar-member-corp,DT Marks Qatar Member Corp,dt-marks-qatar-member-corp,match
20,BEDFORD HILLS CORP,2016,bedford-hills-corp,Bedford Hills Corp,bedford-hills-corp,match
328,TRUMP MARKS EGYPT CORP,2016,trump-marks-egypt-corp,Trump Marks Egypt Corp,trump-marks-egypt-corp,match
120,LFB ACQUISITION LLC,2016,lfb-acquisition-llc,LFB Acquisition LLC,lfb-acquisition-llc,match


In [19]:
# Santardized the slugs within the Entity Relations dataframe
aDf = entityRelDf.copy()
aDf.columns = ['entityA','ownership','entityB','role','year','slug','entityBSlug']

bDf = entityRelDf.copy()
bDf.columns = ['entityA','ownership','entityB','role','year','entityASlug','slug']

mergeA = pd.merge(aDf,entityDf[['year','slug','org_name','org_slug']],on=['year','slug'],how='left')
mergeB = pd.merge(bDf,entityDf[['year','slug','org_name','org_slug']],on=['year','slug'],how='left')

mergeA['entityA'] = mergeA.org_name
mergeA['entityASlug'] = mergeA.org_slug

mergeB['entityB'] = mergeB.org_name
mergeB['entityBSlug'] = mergeB.org_slug

mergeA = mergeA[['entityASlug','entityA','ownership','entityB','entityBSlug','role','year']]
mergeB = mergeB[['entityASlug','entityA','ownership','entityB','entityBSlug','role','year']]

mergeA = mergeA.drop(['entityB','entityBSlug','role','year'],axis=1)
mergeB = mergeB.drop(['entityASlug','entityA','ownership'],axis=1)

entityRelOut = mergeA.copy()
entityRelOut['entityB'] = mergeB.entityB
entityRelOut['entityBSlug'] = mergeB.entityBSlug
entityRelOut['role'] = mergeB.role
entityRelOut['year'] = mergeB.year

entityRelOut.sample(10)

,entityASlug,entityA,ownership,entityB,entityBSlug,role,year
1510,dt-tower-i-llc,DT Tower I LLC,1,DT Tower I Member Corp,dt-tower-i-member-corp,Managing Member,2017
327,tipperary-realty-corp,Tipperary Realty Corp,100,"TRUMP, DONALD J.",trump-donald-j,Shareholder,2016
437,trump-international-hotels-management-llc,Trump International Hotels Management LLC,0.1,TIHM Member Corp,tihm-member-corp,Managing Member,2016
93,dt-home-marks-international-llc,DT Home Marks International LLC,99,"TRUMP, DONALD J.",trump-donald-j,Member,2016
269,thc-miami-restaurant-hospitality-member-corp,THC Miami Restaurant Hospitality Member Corp,100,"TRUMP, DONALD J.",trump-donald-j,Shareholder,2016
1527,thc-jeddah-hotel-advisor-llc,THC Jeddah Hotel Advisor LLC,99,"TRUMP, DONALD J.",trump-donald-j,Member,2017
1494,dt-jeddah-technical-services-manager-member-corp,DT Jeddah Technical Services Manager Member Corp,100,"TRUMP, DONALD J.",trump-donald-j,Shareholder,2017
20,all-county-building-supply-maintenance-corp,All County Building Supply & Maintenance Corp,25,"TRUMP, DONALD J.",trump-donald-j,Shareholder,2016
183,lawrence-towers-apartments-llc,Lawrence Towers Apartments LLC,75,TRUMP FAMILY MEMBERS,trump-family-members,Member,2016
1066,the-east-61-street-company-lp,"East 61 St Company, L.P., The",0.1,Plaza Consulting Corp,plaza-consulting-corp,Partner,2017


### Clean up & save out Entities & Entity Relations

In [21]:
entityDfOut = entityDf.copy().drop(['entity','slug','match_type'],axis=1,inplace=True)
entityDfOut = entityDf[['year','org_slug','org_name']]
display(entityDfOut.sample(10))

entityDfOut.to_csv('data/5_trump-oge278e-exhibit-a-entities.csv',index=False)
entityRelOut.to_csv('data/6_trump-oge278e-exhibit-a-entity-relations.csv',index=False)

,year,org_slug,org_name
228,2016,toronto-development-llc,Toronto Development LLC
476,2017,hwa-555-owners-llc,HWA 555 Owners LLC
369,2017,trump-marks-puerto-rico-i-member-corp,Trump Marks Puerto Rico I Member Corp
90,2017,dt-tower-gurgaon-llc,DT Tower Gurgaon LLC
114,2017,hudson-waterfront-associates-lp,"Hudson Waterfront Associates, L.P."
386,2016,trump-marks-toronto-lp,Trump Marks Toronto LP (Formally Trump Toronto...
125,2016,midland-associates,Midland Associates
57,2016,dt-dubai-golf-manager-llc,DT Dubai Golf Manager LLC
344,2016,trump-marks-llc,Trump Marks LLC
366,2016,trump-marks-products-member-corp,Trump Marks Products Member Corp
